In [1]:
import re

In [2]:
sequence_length = 32

In [3]:
def txt_to_list(path, sequence_length):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
#         text_list = [i for i in text]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
#                 print(entity)            
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
        
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
#             while len(sentence) < sequence_length:
#                 sentence += 'O'
#             if len(sentence) > sequence_length:
#                 sentence = sentence[:sequence_length]
            x.append([i for i in sentence])
            
        tmp = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
#                 while len(tmp) < sequence_length:
#                     tmp.append('O')
#                 if len(tmp) > sequence_length:
#                     tmp = tmp[:sequence_length]
                y.append(tmp)
                tmp = list()
            else:
                tmp.append(i)
        y.append(label_list)
    return [x, y]

In [4]:
x1, y1 = txt_to_list('../datasets/stage1/SampleData_deid.txt', sequence_length)

In [5]:
x2, y2 = txt_to_list('../datasets/stage2/train_1_update.txt', sequence_length)

In [6]:
x4, y4 = txt_to_list('../datasets/stage4/train_2.txt', sequence_length)

In [7]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

76508
76508


In [8]:
# print(x[0])

In [9]:
# print(y[0])

In [10]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

48964 48964
12242 12242
15302 15302


In [11]:
import kashgari
from kashgari.tasks.labeling.models import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding
kashgari.config.use_cudnn_cell = True

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=sequence_length,
                          )

model = BiLSTM_CRF_Model(bert_embed)
model = BiLSTM_CRF_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10,
         )
model.save('model/BiLSTM_CRF_Model_len_32_epo_10')

╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 19)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 19, 100)           173800    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 19, 256)           234496    
_________________________________________________________________
layer_dense (Dense)          (None, 19, 64)            16448     
_________________________________________________________________
layer_crf_dense (Dense)      (None, 19, 31)            2015      
_________________________________________________________________
layer_crf (CRF)              (None, 19, 31)            961       
Total params: 427,720
Trainable params: 427,720
Non-trainable params: 0
_____________________________________________________

In [12]:
model = kashgari.utils.load_model('model/BiLSTM_CRF_Model_len_32_epo_10')
model.evaluate(test_x, test_y)

/home/Danny/anaconda3/lib/python3.6/site-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/home/Danny/anaconda3/lib/python3.6/site-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/home/Danny/anaconda3/lib/python3.6/site-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/home/Danny/anaconda3/lib/python3.6/site-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


                precision    recall  f1-score   support

          time     0.7632    0.7232    0.7426       838
      location     0.9315    0.7640    0.8395        89
      med_exam     0.8333    0.6204    0.7113       137
         money     0.7551    0.7115    0.7327        52
          name     0.8163    0.8000    0.8081       100
    profession     1.0000    0.3333    0.5000         9
            ID     0.3333    0.1250    0.1818         8
       contact     1.0000    0.5385    0.7000        13
     education     0.0000    0.0000    0.0000         1
        family     1.0000    0.7059    0.8276        17
clinical_event     0.0000    0.0000    0.0000         4
  organization     0.0000    0.0000    0.0000         3

     micro avg     0.7879    0.7073    0.7454      1271
     macro avg     0.7862    0.7073    0.7416      1271



'                precision    recall  f1-score   support\n\n          time     0.7632    0.7232    0.7426       838\n      location     0.9315    0.7640    0.8395        89\n      med_exam     0.8333    0.6204    0.7113       137\n         money     0.7551    0.7115    0.7327        52\n          name     0.8163    0.8000    0.8081       100\n    profession     1.0000    0.3333    0.5000         9\n            ID     0.3333    0.1250    0.1818         8\n       contact     1.0000    0.5385    0.7000        13\n     education     0.0000    0.0000    0.0000         1\n        family     1.0000    0.7059    0.8276        17\nclinical_event     0.0000    0.0000    0.0000         4\n  organization     0.0000    0.0000    0.0000         3\n\n     micro avg     0.7879    0.7073    0.7454      1271\n     macro avg     0.7862    0.7073    0.7416      1271\n'

In [13]:
import pandas as pd
df = pd.read_csv('../datasets/stage4/development_2.csv')

In [14]:
def predict_NER(model, text, sequence_length):
    x_list = list()
    while len(text) > sequence_length:
        x_list.append([i for i in text[:sequence_length]])
        text = text[sequence_length:]        
    x_list.append([i for i in text[:sequence_length]])
    y_list = model.predict(x_list)
    for i, j in zip(x_list, y_list):
        print(len(i), len(j))
    merge_list = list()
    for i in y_list:
        for j in i:
            merge_list.append(j)
    return merge_list   

In [15]:
def output_NER(article_id, text, y):
    output = str()
    flag = False
    for i, j in enumerate(y):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            output += '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
    return output

In [16]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for article_id, text in zip(df['article_id'], df['text']):
    y = predict_NER(model, text, sequence_length)
    output += output_NER(article_id, text, y)    

32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
13 13
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 1

32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
29 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 1

32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 1

32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
11 11
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
13 13
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
11 11
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
11 11
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 19
32 1

In [17]:
print(output)

article_id	start_position	end_position	entity_text	entity_type
0	19	22	一般的	name
0	135	138	有感覺	name
0	196	198	道是	med_exam
0	244	247	樣比較	time
0	324	327	師：2	name
0	368	372	說我晚上	time
0	385	387	要吃	time
0	532	535	以前是	name
0	562	565	師：現	name
0	571	574	好。民	name
0	590	593	醫師：	name
0	666	669	時候會	name
0	689	692	師：沒	name
0	700	703	主要是	time
0	831	834	？家屬	name
0	944	947	我聽後	name
0	988	991	民眾：	name
0	1014	1017	。醫師	name
0	1048	1051	。醫師	name
0	1164	1167	照超音	name
0	1235	1238	，可以	name
0	1301	1304	：嘿，	name
0	1317	1320	為這樣	time
0	1329	1330	。	time
0	1358	1361	期的糖	name
0	1382	1386	時候沒有	time
0	1417	1421	腫的有夠	name
0	1426	1429	哦他有	name
0	1507	1510	沒有。	name
0	1551	1554	吐一直	name
0	1558	1562	。醫師：	time
0	1618	1621	天亮。	name
0	1630	1634	去。民眾	location
0	1649	1653	。醫師：	location
0	1702	1705	子。醫	time
0	1753	1756	有錯啦	name
0	1787	1790	子。醫	name
0	1871	1874	。家屬	name
0	1901	1904	：脹氣	name
0	1924	1927	？民眾	name
0	1948	1951	～民眾	name
0	1967	1970	道發炎	time
0	1988	1991	色的那	time
1	181	184	，我們	name
1	268	271	家屬：	name
1	275	278	甚麼都	time

In [18]:
output_path='output/BiLSTM_CRF_Model_len_32_epo_10.tsv'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output)